In [1]:
import sys
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

sys.path.append("/home/webphy/Desktop/dnn_processor/")  # just to enable `dataset`
sys.path.append(
    "/home/webphy/Desktop/dnn_processor/dataset/"
)  # just to enable `dataset`

import tensorflow as tf
import optuna
import dataset
import numpy as np
import keras
import joblib
from datetime import datetime
from keras import layers

2024-02-06 11:46:01.972320: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-06 11:46:01.972344: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-06 11:46:01.973209: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/home/webphy/Desktop/dnn_processor/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DEFAULT_BATCH_SIZE = 64
EPOCHS = 100
DEFAULT_INPUT_SIZE = 128

train_images_rgx = "../../dataset/train_set_4x/*/*.jpeg"
test_images_rgx = "../../dataset/test_set_4x/*/*.jpeg"

train_dataset, test_dataset, num_classes = dataset.create_datasets(
    train_images_rgx, test_images_rgx, DEFAULT_BATCH_SIZE, DEFAULT_INPUT_SIZE
)

In [3]:


def create_model_checkpointer(save_path):
    return keras.callbacks.ModelCheckpoint(
        save_path + "weights/",
        "val_accuracy",
        mode="max",
        save_best_only=True,
        save_weights_only=True,
    )


def create_early_stopper():
    return keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        min_delta=0.05,
        patience=20,
        verbose=0,
        mode="max",
        start_from_epoch=50,
    )

In [5]:
EXP_ID = "manual_test_4x"
batch_size = 64
input_size = 128

# load data
train_dataset, test_dataset, num_classes = dataset.create_datasets(
    train_images_rgx, test_images_rgx, batch_size, input_size
)

x = x_in = layers.Input(shape=(input_size, input_size, 3))

x = keras.layers.Conv2D(4, 4, 4)(x)
x = keras.layers.Activation("relu")(x)
x = keras.layers.Conv2D(16, 3, 2)(x)
x = keras.layers.Activation("relu")(x)
x = keras.layers.Conv2D(5, 1, 1)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(num_classes)(x)

# Build model and optimizer.
model = keras.Model(x_in, x)
assert isinstance(model, keras.Model), "model is not a model"

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 conv2d_3 (Conv2D)           (None, 32, 32, 4)         196       
                                                                 
 activation_2 (Activation)   (None, 32, 32, 4)         0         
                                                                 
 conv2d_4 (Conv2D)           (None, 15, 15, 16)        592       
                                                                 
 activation_3 (Activation)   (None, 15, 15, 16)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 15, 15, 5)         85        
                                                                 
 flatten_1 (Flatten)         (None, 1125)              0   

In [6]:

optimizer = keras.optimizers.Adam()

# compile the model
model.compile(
    optimizer,
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)



In [7]:
# crate model checkpointer
saved_best_model_path = f"saved_models/{EXP_ID}/best_model_t/"
model.save(saved_best_model_path, True, "tf")  # save all model

model_chkp = create_model_checkpointer(saved_best_model_path)
early_stopper = create_early_stopper()

# Training and validating cycle.
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=EPOCHS,
    callbacks=[model_chkp, early_stopper],
    verbose="auto",
)

# load best model
model.load_weights(saved_best_model_path + "weights/")


INFO:tensorflow:Assets written to: saved_models/manual_test_4x/best_model_t/assets


INFO:tensorflow:Assets written to: saved_models/manual_test_4x/best_model_t/assets


Epoch 1/100


I0000 00:00:1707230815.367715  804243 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


55/55 [==============================] - 3s 7ms/step - loss: 1.3541 - accuracy: 0.3486 - val_loss: 1.1039 - val_accuracy: 0.5939
Epoch 2/100
55/55 [==============================] - 0s 5ms/step - loss: 0.8161 - accuracy: 0.6733 - val_loss: 0.7414 - val_accuracy: 0.6820
Epoch 3/100
55/55 [==============================] - 0s 5ms/step - loss: 0.6634 - accuracy: 0.7281 - val_loss: 0.6347 - val_accuracy: 0.7241
Epoch 4/100
55/55 [==============================] - 0s 5ms/step - loss: 0.6131 - accuracy: 0.7591 - val_loss: 0.5917 - val_accuracy: 0.7739
Epoch 5/100
55/55 [==============================] - 0s 5ms/step - loss: 0.5898 - accuracy: 0.7602 - val_loss: 0.6382 - val_accuracy: 0.7241
Epoch 6/100
55/55 [==============================] - 0s 5ms/step - loss: 0.5842 - accuracy: 0.7614 - val_loss: 0.5410 - val_accuracy: 0.7701
Epoch 7/100
55/55 [==============================] - 0s 6ms/step - loss: 0.5323 - accuracy: 0.7866 - val_loss: 0.5231 - val_accuracy: 0.8046
Epoch 8/100
55/55 [======

In [8]:

# Evaluate the model accuracy on the validation set.
score = model.evaluate(test_dataset, verbose=1)

5/5 [==============================] - 0s 3ms/step - loss: 0.3979 - accuracy: 0.8697


In [10]:
model.save(f"saved_models/{EXP_ID}/model.h5")